# Running Tune experiments with ZOOpt

In this tutorial we introduce ZOOpt, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with ZOOpt and, as a result, allow you to seamlessly scale up a ZOOpt optimization process - without sacrificing performance.

Zeroth-order optimization (ZOOpt) does not rely on the gradient of the objective function, but instead, learns from samples of the search space. It is suitable for optimizing functions that are nondifferentiable, with many local minima, or even unknown but only testable. Therefore, zeroth-order optimization is commonly referred to as "derivative-free optimization" and "black-box optimization". In this example we minimize a simple objective to briefly demonstrate the usage of ZOOpt with Ray Tune via `ZOOptSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `zoopt==0.4.1` library is installed. To learn more, please refer to the [ZOOpt website](https://github.com/polixir/ZOOpt).

In [ ]:
# !pip install ray[tune]
!pip install zoopt==0.4.1

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [ ]:
import time

import ray
from ray import tune
from ray.tune.suggest.zoopt import ZOOptSearch
from zoopt import ValueType

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [ ]:
def evaluate(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100) ** (-1) + height * 0.1

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Next we define a search space. The critical assumption is that the optimal hyperparameters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [ ]:
search_config = {
    "steps": 100,
    "width": tune.randint(0, 10),
    "height": tune.quniform(-10, 10, 1e-2),
    "activation": tune.choice(["relu, tanh"])
}

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define the search algorithm built from `ZOOptSearch`, constrained to a maximum of `8` concurrent trials via ZOOpt's internal `"parallel_num"`.

In [ ]:
zoopt_config = {
    "parallel_num": 8
}
algo = ZOOptSearch(
    algo="Asracos",  # only supports ASRacos currently
    budget=num_samples,
    **zoopt_config,
)

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tune.run()`.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    metric="mean_loss",
    mode="min",
    name="zoopt_exp",
    num_samples=num_samples,
    config=search_config
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Optional: passing the parameter space into the search algorithm

We can also pass the parameter space ourselves in the following formats: 
- continuous dimensions: (continuous, search_range, precision)
- discrete dimensions: (discrete, search_range, has_order)
- grid dimensions: (grid, grid_list)

In [ ]:
space = {
    "height": (ValueType.CONTINUOUS, [-10, 10], 1e-2),
    "width": (ValueType.DISCRETE, [0, 10], True),
    "layers": (ValueType.GRID, [4, 8, 16])
}

ZOOpt again handles constraining the amount of concurrent trials with `"parallel_num"`.

In [ ]:
zoopt_search_config = {
    "parallel_num": 8,
    "metric": "mean_loss",
    "mode": "min"
}
algo = ZOOptSearch(
    algo="Asracos",
    budget=num_samples,
    dim_dict=space,
    **zoopt_search_config
)

This time we pass only `"steps"` and `"activation"` to the Tune `config` because `"height"` and `"width"` have been passed into `ZOOptSearch` to create the `search_algo`. 
Again, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    metric="mean_loss",
    mode="min",
    name="zoopt_exp",
    num_samples=num_samples,
    config={
        "steps": 100,
    }
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
ray.shutdown()